# **Procesado Digital de Imagen**

## Lab 4: DFT y Filtros Lineales

2021 - Veronica Vilaplana - [GPI @ IDEAI](https://imatge.upc.edu/web/) Research group

-----------------

En esta práctica se estudian las herramientas para el análisis de imagen en el dominio espacio-frecuencia. Analizaremos el contenido espectral de diferentes imágenes, y veremos los efectos del diezmado y la interpolación.

En primer lugar instalaremos e importaremos las librerías necesarias (es posible que tengas que reiniciar el entorno de trabajo posteriormente).

In [ ]:
# check if this is necessary
!pip install scikit-image pillow plotly

In [ ]:
from skimage import io as skio
from skimage import transform as sktf
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

Definimos funciones para visualizar imágenes y sus transformadas de Fourier.

In [ ]:
def display_image(img, title='', size=None):
  #plt.subplot(1,2,1)
  plt.gray()
  h = plt.imshow(img, interpolation='none')
  if size:
    dpi = h.figure.get_dpi()/size
    h.figure.set_figwidth(img.shape[1] / dpi)
    h.figure.set_figheight(img.shape[0] / dpi)
    h.figure.canvas.resize(img.shape[1] + 1, img.shape[0] + 1)
    h.axes.set_position([0, 0, 1, 1])
    h.axes.set_xlim(-1, img.shape[1])
    h.axes.set_ylim(img.shape[0], -1)
  plt.grid(False)
  plt.title(title)  
  plt.show()

def display_ft(ft, title=''):
  N = ft.shape[0]
  x = np.linspace(-0.5, 0.5, N)
  y = np.linspace(-0.5, 0.5, N)
  X, Y = np.meshgrid(x, y)
  fig = go.Figure(data=[go.Surface(z=ft,x=X,y=Y)])
  fig.update_layout(title=title)
  fig.update_layout(scene = dict(xaxis_title='Fx', yaxis_title='Fy', zaxis_title='FT'))
  fig.show()

def display_dft(ft, title=''):
  fig = go.Figure(data=[go.Surface(z=ft)])
  fig.update_layout(title=title)
  fig.update_layout(scene = dict(xaxis_title='k', yaxis_title='l', zaxis_title='DFT'))
  fig.show()


Carga las imágenes `test1.bmp`, `bottle3.png` y `boat.pgm` en Colab.
Leemos y visualizamos las imágenes para verificar que se han cargado correctamente.

In [ ]:
# Read the files from notebook disk
test1 = skio.imread('test1.bmp')
img = skio.imread('bottle3.png')

display_image(test1, size=1)
display_image(img, size=1)


## 1.	La Transformada Discreta de Fourier y su representación
---


La DFT se calcula con la función NumPy fft2.
Calculamos y mostramos la transformada de la imagen ima. Aprovecharemos la sensibilidad a mayúsculas y minúsculas Python para denominar los espectros con letras mayúsculas.

In [ ]:
IMG = np.fft.fft2(img)


Mostramos el espectro o módulo de la DFT. Necesitamos mostrar los valores en escala logarítmica. Puedes también probar visualizar el resultado con la escala lineal (es decir, sin el log) para ver la diferencia.

In [ ]:
display_image(np.log(1+np.abs(IMG)), title='DFT modulus of IMG', size=1)

En primer lugar, la componente de contínua (o DC) en el dominio transformado de IMA está en el píxel de la esquina superior izquierda de la la transformada visualizada. Recordemos que es espectro de una señal discreta es periódico de periodo $2\pi$ en pulsaciones discretas (1 en frecuencia normalizada) y que la DFT representa con N muestras (si N es el número de puntos de la transformada) un periodo del espectro. En 2D, la transformada representa con NxN muestras un periodo del espectro. De hecho, la matriz transformada IMA que hemos calculado contiene sólo las muestras del período fundamental en $[0,2\pi) x [0,2\pi)$.

Para visualizar transformadas de imagen, se suele situar el origen de frecuencias (componente de continua) en el centro de la imagen. Con ello se aprecia mejor la simetría conjugada. Para ello, empleamos la función fftshift()

Por otra parte, como mencionamos anteriormente, al ser tan alto el valor de la componente continua, se enmascara la lectura de la información restante. Se impone por lo tanto el uso de una escala logarítmica sobre la transformada.

In [ ]:
IMG_mod=np.fft.fftshift(np.log(1+np.abs(IMG)))
display_image(IMG_mod, title='Centered DFT modulus of IMG', size=1)

<font color='blue'>Pregunta: Relaciona las estructuras que aparecen en el espectro con formas de la imagen original.</font>

---
<font color='red'>Respuesta: 

</font> 

Repite el proceso para generar el módulo de la transformada de la imagen `test1`. Utiliza el espacio inferior para escribir los comandos necesarios. Análogamente al caso anterior, utiliza las mayúsculas para denominar TEST1 a la transformada y TEST1_mod a su módulo. 

In [ ]:
# Compute and display test1 DFT


<font color='blue'>Pregunta: Relaciona las estructuras que aparecen en el espectro con formas de la imagen original.</font>

---
<font color='red'>Respuesta: 

</font> 

Repite el proceso anterior para generar el módulo de la transformada de la imagen `boat3`. Utiliza el espacio inferior para escribir los comandos MATLAB necesarios. Análogamente al caso anterior, utiliza las mayúsculas para denominar BOAT a la transformada y BOAT_mod a su módulo

In [ ]:
# Compute and display boat3 DFT

<font color='blue'>Pregunta: Relaciona las estructuras que aparecen en el espectro con formas de la imagen original.</font>

---
<font color='red'>Respuesta: 

</font> 

## 2. Diezmado e interpolación

### Diezmado y réplicas espectrales
---
El teorema de Nyquist establece que antes de diezmar o submuestrear una imagen es preciso limitar su ancho de banda a la mitad de la nueva frecuencia de muestreo. De no hacerlo, pueden aparecer problemas de aliasing. No obstante, vamos a diezmar directamente las imágenes y analizar su espectro tras el diezmado.

Trabajaremos con la imagen `test1` que utilizamos en el apartado anterior, y con la imagen `bottle3`.
Utilizaremos la función [resize](http://scikit-image.org/docs/stable/api/skimage.transform.html#resize) de la librería skimage.
Para diezmar la imagen por un factor de 3 haremos lo siguiente.

Puedes utilizar el parametro `size`en la función de visualización (`display_image`) para hacer un zoom de la figura replicando píxeles.


In [ ]:
from skimage import transform

img_sub = transform.resize(img,np.asarray(img.shape)/3, anti_aliasing=False, mode='constant');
display_image(img_sub,title='img resized', size=1)
display_image(img_sub,title='img resized and zoomed', size=3)
display_image(img, title='img', size=1)


In [ ]:
test1_sub=transform.resize(test1,np.asarray(test1.shape)/3, anti_aliasing=False, mode='constant');
display_image(test1_sub, title='test1 resized', size=1)
display_image(test1_sub, title='test1 resized and zoomed', size=3)
display_image(test1, title='test1', size=1)

<font color='blue'>Pregunta: ¿Puedes interpretar los resultados? ¿Aparece algún efecto especial? Intenta explicar el efecto producido al visualizar la versión diezmada por 3 de la imagen `test1`.</font>

---
<font color='red'>Respuesta: 

</font>


Veamos el espectro de las imágenes antes y después del submuestreo:

In [ ]:
IMG_sub_mod = np.fft.fftshift(np.log(1+np.abs(np.fft.fft2(img_sub))))
display_image(IMG_mod, title='DFT Modulus IMG', size=1)
display_image(IMG_sub_mod, title='DFT Modulus IMG resized', size=1)

In [ ]:
TEST1_mod = np.fft.fftshift(np.log(1+np.abs(np.fft.fft2(test1)))) # DELETE
TEST1_sub_mod = np.fft.fftshift(np.log(1+np.abs(np.fft.fft2(test1_sub))))
display_image(TEST1_mod, title='DFT modululs TEST1', size=1)
display_image(TEST1_sub_mod, title='DFT modulus TEST1 resized', size=3)

De hecho, las transformadas que estamos visualizando representan sólo una parte del espectro de las de las imágenes originales tras el diezmado (recuerda que tanto el espectro como la imagen son periódicos).
Vamos a manipular las matrices de la transformada para visualizar varias réplicas espectrales yuxtapuestas de modo de generar el efecto de periodicidad,
lo cual nos dará una idea más aproximada del espectro de las imágenes submuestreadas:

In [ ]:
display_image(IMG_mod, title='DFT modulus IMG', size=1)
tmp = np.hstack((IMG_sub_mod,IMG_sub_mod,IMG_sub_mod))
IMG_sub_mod_replicas = np.vstack((tmp,tmp,tmp))
display_image(IMG_sub_mod_replicas,title='DFT modulus IMG resized (replicas)',size=1)

In [ ]:
display_image(TEST1_mod, title='DFT modulus TEST1', size=1)
tmp = np.hstack((TEST1_sub_mod,TEST1_sub_mod,TEST1_sub_mod))
TEST1_sub_mod_replicas = np.vstack((tmp,tmp,tmp))
display_image(TEST1_sub_mod_replicas,title='DFT modulus TEST1 resized (replicas)',size=1)

## 3. Filtrado en frecuencia

La forma directa de filtrar una imagen es calcular la convolución de la imagen con la respuesta al impulso del filtro, como hicimos en el Laboratorio 3.
Otra forma de filtrar una imagen en el dominio de la frecuencia consiste en eliminar algunos coeficientes transformados correspondientes a las componentes frecuenciales que queremos eliminar.

Sin embargo, esta técnica de filtrado en frecuencia mediante una máscara de frecuencias no siempre es adecuada. Vamos a explorar esta observación.

Supongamos que se quieren evitar problemas de aliasing en el diezmado por un factor de 2 de la imagen `test1`.

Recordemos los efectos del diezmado sin filtrar.


In [ ]:
display_image(test1,title='test1',size=1)
test1_sub=transform.resize(test1,np.asarray(test1.shape)/2, anti_aliasing=False, mode='constant');

display_image(test1_sub, title='test1 resized', size=2)

Podríamos haber eliminado los coeficientes superiores a la mitad de la nueva frecuencia de muestreo antes del diezmado, empleando una máscara, así:

Observa que la transformada TEST1 no está centrada (auqnue sí lo está al hacer el display) 


In [ ]:
TEST1 = np.fft.fft2(test1)
TEST1_mod = np.fft.fftshift(np.log(1+np.abs(TEST1)))
fils = TEST1.shape[0]
cols = TEST1.shape[1]

print(cols/6)
# mask vertical frequencies
TEST1[int(1/6*fils):int(5/6*fils)+1,:] = 0;

# mask horizontal frequencies
TEST1[:,int(1/6*cols):int(5/6*cols)+1] = 0;

display_image(TEST1_mod, title='DFT modulus TEST1', size=1)
display_image(np.fft.fftshift(np.log(1+np.abs(TEST1))), title='DFT modulus TEST1 filtered', size=1)


Para ver la versión filtrada de la imagen, debemos aplicar la FFT inversa:


In [ ]:
test1_filt = 9*np.real(np.fft.ifft2(TEST1))
display_image(test1, title='test1', size=1)
display_image(test1_filt, title='test1 (filtered)', size=1)

Comparemos ahora el resultado que se obtiene al realizar el submuestreo de la imagen filtrada con el resultado anterior:

In [ ]:
test1_sub  = transform.resize(test1,     np.asarray(test1.shape)/3,      anti_aliasing=False, mode='constant');
test1_sub2 = transform.resize(test1_filt,np.asarray(test1_filt.shape)/3, anti_aliasing=False, mode='constant');

display_image(test1_sub,title='test1 downsampled (no filter)', size=1)
display_image(test1_sub2,title='test1 downsampled (filtered)', size=1)


<font color='blue'>Pregunta: Comenta las diferencias entre una y otra imagen </font>

----
<font color='red'>Respuesta:
</font>


Verás que, esta vez, no se ha producido aliasing. No obstante, esta técnica de filtrado no siempre da los resultados deseados. Vamos a aplicar la misma técnica a la imagen `img`. Y la filtramos como anteriormente hicimos con `test2`:

In [ ]:
IMG = np.fft.fft2(img)
IMG_mod = np.fft.fftshift(np.log(1+abs(IMG)))
fils = IMG.shape[0]
cols = IMG.shape[1]
# mask vertical frequencies
IMG[int(1/6*fils):int(5/6*fils)+1,:] = 0
# mask horizontalal frequencies
IMG[:,int(1/6*cols):int(5/6*cols)+1] = 0

display_image(IMG_mod, title='DFT modulus IMG', size=1)
display_image(np.fft.fftshift(np.log(1+np.abs(IMG))), title='DFT modulus IMG (filtered)', size=1)

Veamos ahora el resultado del filtrado mediante la FFT inversa:

In [ ]:
img_filt = 9*np.real(np.fft.ifft2(IMG));
display_image(img, title='img', size=1)
display_image(img_filt, title='img (filtered)', size=1)

<font color='blue'>Pregunta: ¿Observas algún efecto especial en la imagen filtrada? ¿Podrías explicarlo? 
</font>

---
<font color='red'>Respuesta: 

</font>

## 4. Diezmado e interpolación con filtros paso-bajo

Evidentemente, los efectos de aliasing que han aparecido al diezmar, podrían haberse evitado empleando filtros más sencillos que la técnica de filtrado en frecuencia que se ha propuesto. Veamos qué hubiera ocurrido con un simple filtrado gaussiano.


Primero diezmamos directamente la imagen `test1`


In [ ]:
test1_sub  = transform.resize(test1, np.asarray(test1.shape)/3, anti_aliasing=False, mode='constant');

Y a continuación diezmamos después de filtrar:


In [ ]:
from skimage import filters

sigma = 1
window = 5
truncate = (((window - 1)/2)-0.5)/sigma
test1_filt = filters.gaussian(test1,sigma=sigma,truncate=truncate);
test1_sub2 = transform.resize(test1_filt,np.asarray(test1_filt.shape)/3, anti_aliasing=False, mode='constant');

In [ ]:
display_image(test1,title='test1 (original)', size=1)
display_image(test1_sub,title='test1 downsampled (no filter)', size=1)
display_image(test1_sub2,title='test1 downsampled (filtered)', size=1)

Repetimos para la imagen img

In [ ]:
img_sub  = transform.resize(img, np.asarray(img.shape)/3, anti_aliasing=False, mode='constant');
img_filt = filters.gaussian(img,sigma=sigma,truncate=truncate);
img_sub2 = transform.resize(img_filt,np.asarray(img_filt.shape)/3, anti_aliasing=False, mode='constant');


In [ ]:
display_image(img,title='img',size=1)
display_image(img_sub, title='img downsampled (no filter)', size=1)
display_image(img_sub2, title='img downsampled (filtered)', size=1)

<font color='blue'>Pregunta: Comenta los resultados 
</font>

---
<font color='red'>Respuesta:

## 5. Comparación de un filtro paso-bajo y uno paso-alto 

Finalmente compararemos el funcionamiento de un filtro paso-bajo y otro paso-alto.

Utilizaremos la imagen `boat`


In [ ]:
img2 = skio.imread('boat.pgm')
display_image(img2, size=1)

Primero generamos la respuesta al impulso del filtro Gaussiano y visualizamos su respuesta frecuencial

In [ ]:
def gaussian_kernel(size=5,sigma=1.0):
  a = np.zeros((size, size))
  a[int(np.floor(size/2)),int(np.floor(size/2))] = 1
  h = filters.gaussian(a, sigma=sigma, mode='constant', cval=0)
  return h/np.sum(h)

# Define a gaussian kernel (size=5 and sigma=1)
h_gau = gaussian_kernel(size=5,sigma=1)
print(h_gau)

# Visualize its frequency response
N = 256
H_GAU = np.abs(np.fft.fftshift(np.fft.fft2(h_gau,(N,N))))
display_ft(H_GAU)

Hacemos lo mismo para el filtro laplaciano:

In [ ]:
def laplace_kernel(alpha=0.0):
  h = np.zeros((3, 3))
  h1    = alpha/(alpha+1); h2 = (1-alpha)/(alpha+1);
  h[0,:] = [h1,h2,h1]
  h[1,:] = [h2, -4/(alpha+1), h2]
  h[2,:] = [h1,h2,h1]
  return h

# Define a laplacian kernel
h_lap = laplace_kernel()
print(h_lap)

# Visualize its frequency response
N = 256
H_LAP = np.abs(np.fft.fftshift(np.fft.fft2(h_lap,(N,N))))
display_ft(H_LAP)


Ahora calculamos la convolución 2D entre la imagen y la respuesta al impulso del filtro.

In [ ]:
import scipy
img2_gau = scipy.signal.convolve2d(img2,h_gau,mode='same')
img2_lap = scipy.signal.convolve2d(img2,h_lap,mode='same')

display_image(img2,size=1, title='Original')
display_image(img2_gau,size=1, title='Gaussian filter')
display_image(img2_lap,size=1, title='Laplacian filter')

<font color='blue'>Pregunta: Comenta y compara las respuestas frecuenciales y el efecto de aplicar ests filtros </font>

---
<font color='red'>Respuesta:

Cuál es el valor medio de las imágenes filtradas? Observa que la función ´display_image´ normaliza la visualización de la imagen entre sus valores mínimo (negro) y máximo (blanco).

In [ ]:
print('mean value original image = ', np.mean(img2))
print('mean value gaussian filtered image = ', np.mean(img2_gau))
print('mean value laplacian filtered image = ', np.mean(img2_lap))

<font color='blue'> Pregunta: Explica cómo modifica el valor medio de la imagen cada uno de estos filtros.

Relaciona el cambio en el valor medio de la imagen con alguna característica espcífica de la respuesta frecuencial o la respuesta al impulso del filtro.</font>

----
<font color='red'> Respuesta: 
</font>
